In [1]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pydantic transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [13]:
!ngrok config add-authtoken 2ghaBW9LmLOSTQbsBcaXRfDUavI_6aMAAhus477xme7eWbgvP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [15]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

app = FastAPI()

class TextInput(BaseModel):
    text: str

# Load the Hugging Face model and tokenizer
model_name = "Seethal/sentiment_analysis_generic_dataset"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the sentiment analysis endpoint
@app.post("/analyze")
async def analyze_text(input_text: TextInput):
    # Validate input text length
    MAX_TEXT_LENGTH = 1000
    if len(input_text.text) > MAX_TEXT_LENGTH:
        raise HTTPException(status_code=400, detail="Text length exceeds maximum allowed length")

    # Preprocess the input text
    inputs = tokenizer(input_text.text, return_tensors="pt")
    outputs = model(**inputs)
    sentiment_scores = torch.nn.functional.softmax(outputs.logits, dim=1)

    # Get the sentiment label with the highest score
    sentiment_label = torch.argmax(sentiment_scores)

    # Return the sentiment label and score
    return {"sentiment": sentiment_label.item(), "score": sentiment_scores[0][sentiment_label].item()}

# Define the welcome endpoint
@app.get("/")
async def welcome():
    return "Welcome to our Text Classification API"

In [16]:
import requests

url = "https://sentiment-analysis9.p.rapidapi.com/sentiment"
user_input= input("Please enter a text for sentiment analysis:")

def call_sentiment_api(user_input):
  payload = [
	{
		"id": "1",
		"language": "en",
		"text": "I love the service"
	}
]
  headers = {
	"content-type": "application/json",
	"Accept": "application/json",
	"X-RapidAPI-Key": "SIGN-UP-FOR-KEY",
	"X-RapidAPI-Host": "sentiment-analysis9.p.rapidapi.com"
}

  response = requests.post(url, json=payload, headers=headers)

  print(response.json())
  return response.json()

@app.get("/")
async def welcome():
    return "Welcome to our Text Classification API"

@app.get('/2nd')
async def index():
 return {'message':'Hello, I am a developer'}

@app.get('/{name}')
async def get_name(name: str):
 return {'Welcome To Here': f'{name}'}


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Please enter a text for sentiment analysis:She is sad today
Public URL: https://9e2e-35-243-129-98.ngrok-free.app


INFO:     Started server process [297]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     58.65.195.141:0 - "GET / HTTP/1.1" 200 OK
INFO:     58.65.195.141:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     58.65.195.141:0 - "GET /2nd HTTP/1.1" 200 OK
INFO:     58.65.195.141:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     58.65.195.141:0 - "GET /%7BMAdiha%7D HTTP/1.1" 200 OK
INFO:     58.65.195.141:0 - "GET /favicon.ico HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [297]


KeyboardInterrupt: 

In [17]:
# Load the pre-trained model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

@app.post("/sentiment")
async def analyze_sentiment(text: str):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")

    # Get the sentiment score
    with torch.no_grad():
        output = model(**inputs)
        scores = output[0][0].detach().numpy()
         # Interpret the sentiment score
    sentiment = "positive" if scores > 0 else "negative"

    return {"sentiment": sentiment}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, port=8000)

INFO:     Started server process [297]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [297]


KeyboardInterrupt: 